# Evaluate the box below to initialize the web UI.

In [ ]:
import os
import pprint
import ipywidgets as widgets
import ray
import subprocess
import tempfile

from IPython.display import display

ray.init(redis_address=os.environ["REDIS_ADDRESS"])

**Evaluate the box below to search for objects.**

In [ ]:
object_search = widgets.Text(
    value="",
    placeholder="Object ID",
    description="Search for an object:",
    disabled=False
)
display(object_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.object_table(object_search.value))

object_search.on_submit(handle_submit)

**Evaluate the box below to search for tasks.**

In [ ]:
task_search = widgets.Text(
    value="",
    placeholder="Task ID",
    description="Search for a task:",
    disabled=False
)
display(task_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.task_table(task_search.value))

task_search.on_submit(handle_submit)

#### Task timeline.

In [ ]:
path_input = widgets.Button(description="View task timeline")
display(path_input)

def find_trace2html():
    trace2html = os.path.join(os.path.expanduser("~"), "catapult", "tracing", "bin", "trace2html")
    assert os.path.exists(trace2html), "Could not find catapult, please clone it into your home directory from https://github.com/catapult-project/catapult/tree/master/tracing"
    return trace2html

def handle_submit(sender):
    tmp = tempfile.mktemp() + ".json"
    tmp2 = tempfile.mktemp() + ".html"

    print("Dumping task profiling data to " + tmp)
    ray.global_state.dump_catapult_trace(tmp)
    print("Converting chrome trace to " + tmp2)
    trace2html = find_trace2html()
    subprocess.check_output(["python2", trace2html, tmp, '--output', tmp2])
    print("Opening html file in browser...")
    subprocess.Popen(["open", "-a", "Google Chrome", tmp2])

path_input.on_click(handle_submit)